## Vanilla RAG using DsPy

This code demonstrates the creation of a Vanilla RAG (Retrieval-Augmented Generation) model using the DSPy library in Python. Given the limited time frame of a couple of days, the implementation is functional but leaves room for optimization and experimentation across several areas:

- Document Loading: Beyond the PyPDFLoader from LangChain, other libraries like PyPDF2, PDFMiner, Tabula, PDFQuery, or LlamaParse could be explored for potentially better performance or compatibility with different document types.

- Chunking Strategy: LangChain offers various text splitters, and adjusting parameters like chunk size, overlap, and separators could enhance the efficiency of the model's retrieval process.

- Embeddings: Different embeddings can be experimented with, balancing memory usage, processing time, and accuracy to better suit the specific use case.

- Vector Database: Exploring alternatives such as Faiss, ChromaDB, or others might yield improvements in retrieval speed and accuracy.

- LLM Model: Experimentation with different language models, such as Llama, GPT, Claude, or Gemini, could lead to better generation quality

- Retrieval Strategy: Adjusting the number of relevant chunks retrieved or modifying the retrieval algorithm could optimize the relevance and coherence of generated answers.

- Training and Testing Dataset: Expanding the dataset with more diverse samples can significantly enhance the model's robustness and generalizability.

- Optimization Techniques: Implementing custom optimization functions tailored to the specific requirements of the task could further refine the model's performance.

- Evaluation Metrics: Developing custom evaluation metrics aligned with the project’s goals would provide more meaningful insights into model performance.

- Pipeline Configuration: The sequence and combination of modules in the pipeline can be adjusted for better synergy between retrieval and generation stages.

This code forms a solid foundation but with further refinement and testing, the RAG model can be significantly optimized for better results.

In [2]:
# pip install --upgrade typing_extensions
# pip install --upgrade dspy
# pip install langchain
# pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# pip install openai==0.28.1
# pip install pypdf
# pip install chromadb
# pip install qdrant_client
# pip install dspy-ai[qdrant]
# pip install dspy-ai\[qdrant\]

In [3]:
import time
import openai
import pandas as pd
import numpy as np
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import dspy

/Users/aa813076/anaconda3/envs/env-webref/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


#### Loading the data

In [4]:
# Defining the textsplitter for creating chunks and loading the data into the vector database
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    length_function = len,
    is_separator_regex = False,
)
docs = PyPDFLoader("GST Smart Guide.pdf").load_and_split(text_splitter = text_splitter)

doc_contents = [doc.page_content for doc in docs]
# List to hold the IDs for each document
doc_ids = list(range(0, len(docs)))

# Embeddings
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device ='cpu')

/Users/aa813076/anaconda3/envs/env-webref/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# vectors = model.encode(doc_contents) # Do this step once and store the results in .npy file and load it from there, to save time

# # Save vectors to a .npy file
# np.save('vectors.npy', vectors)

In [5]:
# Load vectors from a .npy file
vectors = np.load('vectors.npy')

In [6]:
len(vectors), len(vectors[0])

(4245, 768)

In [7]:
# Initiating the vector database - Qdrant
from qdrant_client import QdrantClient
client = QdrantClient(":memory:")

In [8]:
from qdrant_client.models import Distance, VectorParams

# Delete the database if already exists
client.delete_collection(collection_name = "gst_data") 

client.create_collection(
    collection_name = "gst_data",
    vectors_config = VectorParams(size = 768, distance = Distance.COSINE),
)

True

In [9]:
client.upload_collection(
    collection_name = "gst_data",
    ids =  doc_ids,
    vectors = vectors,
    # payloads = [{"text": doc_text} for doc_text in doc_contents]  # Assuming `documents` is a list of texts
)

#### Checking if the vectorDB is loaded correctly

In [10]:
# Get the total number of vectors in the collection
collection_info = client.get_collection(collection_name = "gst_data")
vector_count = collection_info.points_count

print(f"Total number of vectors in the collection: {vector_count}")

# Fetch specific vectors by their IDs
response = client.retrieve(
    collection_name = "gst_data",
    ids = doc_ids[:5],  # Fetch the first 5 vectors, for example
)

# Check if vectors were retrieved successfully
for idx, vector in enumerate(response):
    print(f"Vector ID: {doc_ids[idx]} retrieved: {vector is not None}")

Total number of vectors in the collection: 4245
Vector ID: 0 retrieved: True
Vector ID: 1 retrieved: True
Vector ID: 2 retrieved: True
Vector ID: 3 retrieved: True
Vector ID: 4 retrieved: True


#### Loading the LLM

In [13]:
# LM Model
AZURE_OPENAI_MODEL_NAME = 'openai-gpt4'
AZURE_ENDPOINT = "api-base"
API_VERSION = "2023-05-15"
API_KEY = "api-key"

lm_model = dspy.AzureOpenAI(deployment_id = AZURE_OPENAI_MODEL_NAME, 
                    api_key = API_KEY,         
                    api_base = AZURE_ENDPOINT,
                    api_version = API_VERSION)

dspy.configure(lm = lm_model)

# Checking the model loading 
print(lm_model("hello! this is a raw prompt to GPT-4"))

# Example DSPy Chain of Thought Question-Answer program
qa = dspy.ChainOfThought('question -> answer')
response = qa(question = "What is the capital of France?") #Prompted to lm_model
print(response.answer)

['Hello! How can I assist you today?']
Paris


#### Retriever model

In [21]:
#pip install --upgrade dspy

In [22]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("gst_data", client, k = 5)

# Configuring the DSPy Module
dspy.settings.configure(rm = qdrant_retriever_model, lm = lm_model)

In [24]:
# Checking the retriever model 
def get_context(query):
    query_vector = model.encode(query)
    hits = client.search(
        collection_name = "gst_data",
        query_vector = query_vector,
        limit = 5
    )
    context = []
    for i in hits:
        context.append(doc_contents[i.id])
    return context

query = "What is GST?"
get_context(query)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


['2 GST Smart Guide  Chap. 1 \n principal causes of the prosperity of Great Britain; every great country being \nnecessarily the best and most extensive market for the greater part of the \nproductions of its own industry. If the same freedom, in consequence of the same \nuniformity, could be extended to Ireland and the plantations, both the grandeur \nof the state and the prosperity of every part of the empire, would probably be still \ngreater than at present” \n— Adam Smith in ‘Wealth of Nations’ \n1. Introductory \nGoods and Services Tax, which is better known by its acronym, GST, is the \nbiggest indirect tax reform in India, since independence. GST introduces a single \ntax within the federal structure of our nation. It is a tax on the supply of goods \nand services, levied at every point of consumption of goods or services, from the \nstage of the manufacturer to the final stage of consumer. GST is a destination \nbased tax and is based on the principles of Value Added Taxation.

In [25]:
# Building signatures
class GenerateAnswer(dspy.Signature):
    """Answer questions with short answers."""

    context =  dspy.InputField(desc = "question -> answer")
    question = dspy.InputField()
    answer = dspy.OutputField(desc = "Relevant answer")

In [26]:
# Building the Pipeline
class RAG(dspy.Module):
    def __init__(self, num_passages = 5):
        super().__init__()
        self.retrieve = dspy.Retrieve(k = num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer, temperature = 0.4)

    def forward(self, question):
        context = get_context(question)
        # print(context)
        prediction = self.generate_answer(context = context, question = question)
        return dspy.Prediction(context = context, answer = prediction.answer)


In [27]:
rag = RAG()
def respond(query):
    response = rag(query)
    return response

In [28]:
print(respond(query))

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.34it/s]


Prediction(
    context=['2 GST Smart Guide  Chap. 1 \n principal causes of the prosperity of Great Britain; every great country being \nnecessarily the best and most extensive market for the greater part of the \nproductions of its own industry. If the same freedom, in consequence of the same \nuniformity, could be extended to Ireland and the plantations, both the grandeur \nof the state and the prosperity of every part of the empire, would probably be still \ngreater than at present” \n— Adam Smith in ‘Wealth of Nations’ \n1. Introductory \nGoods and Services Tax, which is better known by its acronym, GST, is the \nbiggest indirect tax reform in India, since independence. GST introduces a single \ntax within the federal structure of our nation. It is a tax on the supply of goods \nand services, levied at every point of consumption of goods or services, from the \nstage of the manufacturer to the final stage of consumer. GST is a destination \nbased tax and is based on the principles 

#### Checking model performance

The train and test datasets are created by randomly putting together some 15 questions, they should not be considered as a ground truth for evaluating the model. 

In [30]:
import json

testdata = json.load(open("TestData.json", "r"))['examples']
testset = [dspy.Example(question = e['question'], answer = e['answer']).with_inputs('question') for e in testdata]

In [31]:
rag = RAG()
from dspy.evaluate.evaluate import Evaluate
evaluate_on_qa = Evaluate(devset = testset, 
                          num_threads = 1, 
                          display_progress = True, 
                          display_table = 17)

# Evaluating using the predefined metric by dspy
metric = dspy.evaluate.answer_exact_match 
evaluate_on_qa(rag, metric = metric)

  0%|          | 0/7 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 19.91it/s]:36<00:06,  6.92s/it]


Backing off 0.4 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}
Backing off 1.7 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}
Backing off 3.8 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}
Backing off 4.6 seconds after 4 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}
Backing off 6.3 seconds after 5 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}
Backing off 6.1 seconds after 6 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.4}


Average Metric: 0 / 7  (0.0): 100%|██████████| 7/7 [01:10<00:00, 10.03s/it]
/Users/aa813076/anaconda3/envs/env-webref/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


Average Metric: 0 / 7  (0.0%)


,question,example_answer,context,pred_answer,answer_exact_match
0,What is MSME?,"MSME stands for Micro, Small and Medium Enterprises","['Appx. 11 Micro, Small and Medium Enterprises 1203 \n (i) Evaluate readiness of MSMEs to export their products and services \n (ii) Recognize areas where...","Micro, Small and Medium Enterprises",False
1,What are the benefits to MSMEs under GST?,"The Micro, Small and Medium Enterprises have been accorded with lot of benefits in terms of compliance reliefs given in the form of threshold exemptions,...","['be treated as deemed accepted by him. \n(9) EWB can be generated online on https://www. ewaybillgst.gov.in. In \naddition to web, EWB can be generated by...","The benefits to MSMEs under GST include compliance reliefs like threshold exemptions, the Composition levy scheme, and quarterly filing of GST returns.",False
2,What is the role of the GST Network (GSTN)?,"GSTN would provide three front end services to the taxpayers namely registration, payment and return. It would be developing back-end IT modules for 27 States...",['Chap. 1 GST — Concept & Status 31 \n 27. Goods & Services Tax Network \n27.1 Goods and Services Tax Network (GSTN) has been set...,The role of the GST Network (GSTN) is to provide front-end services to taxpayers,False
3,What is the late fees per day for not filing Anual GST returns?,The penalty for not filing GST returns is ₹100 per day for CGST and ₹100 per day for SGST,"['976 GST Smart Guide Chap. 54 \n• Amount of pre-deposit payable for filing of appeal under the CGST \nAct, 2017 before the Appellate Authority and...",`100 per day,False
4,What is the GST compensation cess?,GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.,"['of the Council, for the purposes of providing compensation to the States for \nloss of revenue arising on account of implementation of the goods and...","The GST compensation cess is an additional levy on certain notified goods, over and above the applicable GST, aimed at compensating states for revenue loss...",False
5,What are the components of GST?,"The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).",['2 GST Smart Guide Chap. 1 \n principal causes of the prosperity of Great Britain; every great country being \nnecessarily the best and most extensive...,"CGST, SGST/UTGST, and IGST",False
6,What is a non-resident taxable person under GST?,A non-resident taxable person is someone who occasionally supplies goods or services in India without having a fixed place of business and is liable to...,"['(g) any casual taxable \nperson located in the \ntaxable territory.', 'various taxable person as under:', 'registration indicated in his application of registration, an application in...",A non-resident taxable person under GST is any person who occasionally undertakes transactions involving the supply of goods or services or both.,False


0.0

#### Optimize

In [32]:
from dspy import teleprompt

def validate_context_and_answer(example, pred, trace = None):
    if pred.context is None:
        return False
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = teleprompt.BootstrapFewShot(metric = validate_context_and_answer)

trainset = json.load(open("TrainData.json", "r"))['examples']
trainset = [dspy.Example(question = e['question'], answer = e['answer']).with_inputs('question') for e in testdata]
compiled_rag = teleprompter.compile(RAG(), trainset = trainset)

Batches: 100%|██████████| 1/1 [00:00<00:00, 21.40it/s]


Backing off 0.2 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 0.7 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 3.4 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 6.3 seconds after 4 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 15.4 seconds after 5 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}


100%|██████████| 7/7 [01:27<00:00, 12.46s/it]

Bootstrapped 1 full traces after 7 examples in round 0.


In [33]:
metric = dspy.evaluate.answer_exact_match
evaluate_on_qa(compiled_rag, metric = metric)

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.36it/s]


Backing off 0.3 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 1.2 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 2.4 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 1.0 seconds after 4 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 0.8 seconds after 5 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 30.9 seconds after 6 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.92it/s]00:47<04:47, 47.91s/it]


Backing off 0.9 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 1.9 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}
Backing off 3.7 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {'temperature': 0.0, 'max_tokens': 75, 'n': 1}


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.06it/s]1:03<02:24, 28.88s/it] 


...

Average Metric: 3 / 7  (42.9): 100%|██████████| 7/7 [05:12<00:00, 44.65s/it]

Average Metric: 3 / 7  (42.9%)



/Users/aa813076/anaconda3/envs/env-webref/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' 'False' 'False' '✔️ [True]' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,context,pred_answer,answer_exact_match
0,What is MSME?,"MSME stands for Micro, Small and Medium Enterprises","['Appx. 11 Micro, Small and Medium Enterprises 1203 \n (i) Evaluate readiness of MSMEs to export their products and services \n (ii) Recognize areas where...","MSME stands for Micro, Small, and Medium Enterprises.",✔️ [True]
1,What are the benefits to MSMEs under GST?,"The Micro, Small and Medium Enterprises have been accorded with lot of benefits in terms of compliance reliefs given in the form of threshold exemptions,...","['be treated as deemed accepted by him. \n(9) EWB can be generated online on https://www. ewaybillgst.gov.in. In \naddition to web, EWB can be generated by...","The benefits to MSMEs under GST include compliance reliefs such as threshold exemptions, the Composition levy scheme, and the provision for quarterly filing of GST...",False
2,What is the role of the GST Network (GSTN)?,"GSTN would provide three front end services to the taxpayers namely registration, payment and return. It would be developing back-end IT modules for 27 States...",['Chap. 1 GST — Concept & Status 31 \n 27. Goods & Services Tax Network \n27.1 Goods and Services Tax Network (GSTN) has been set...,"The role of the GST Network (GSTN) is to provide front-end services to taxpayers, including registration, payment, and return filing, and to develop back-end IT...",False
3,What is the late fees per day for not filing Anual GST returns?,The penalty for not filing GST returns is ₹100 per day for CGST and ₹100 per day for SGST,"['976 GST Smart Guide Chap. 54 \n• Amount of pre-deposit payable for filing of appeal under the CGST \nAct, 2017 before the Appellate Authority and...",The late fees per day for not filing Annual GST returns is ₹200 (₹100 for CGST and ₹100 for SGST).,False
4,What is the GST compensation cess?,GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.,"['of the Council, for the purposes of providing compensation to the States for \nloss of revenue arising on account of implementation of the goods and...",GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.,✔️ [True]
5,What are the components of GST?,"The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).",['2 GST Smart Guide Chap. 1 \n principal causes of the prosperity of Great Britain; every great country being \nnecessarily the best and most extensive...,"The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).",✔️ [True]
6,What is a non-resident taxable person under GST?,A non-resident taxable person is someone who occasionally supplies goods or services in India without having a fixed place of business and is liable to...,"['(g) any casual taxable \nperson located in the \ntaxable territory.', 'various taxable person as under:', 'registration indicated in his application of registration, an application in...",A non,False


42.86

In [34]:
# Checking the model performance on a different metrics
metric = dspy.evaluate.answer_passage_match
evaluate_on_qa(compiled_rag, metric = metric)

Average Metric: 2 / 7  (28.6): 100%|██████████| 7/7 [00:00<00:00, 18.55it/s]

Average Metric: 2 / 7  (28.6%)



/Users/aa813076/anaconda3/envs/env-webref/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' '✔️ [True]' 'False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,context,pred_answer,answer_passage_match
0,What is MSME?,"MSME stands for Micro, Small and Medium Enterprises","['Appx. 11 Micro, Small and Medium Enterprises 1203 \n (i) Evaluate readiness of MSMEs to export their products and services \n (ii) Recognize areas where...","MSME stands for Micro, Small, and Medium Enterprises.",✔️ [True]
1,What are the benefits to MSMEs under GST?,"The Micro, Small and Medium Enterprises have been accorded with lot of benefits in terms of compliance reliefs given in the form of threshold exemptions,...","['be treated as deemed accepted by him. \n(9) EWB can be generated online on https://www. ewaybillgst.gov.in. In \naddition to web, EWB can be generated by...","The benefits to MSMEs under GST include compliance reliefs such as threshold exemptions, the Composition levy scheme, and the provision for quarterly filing of GST...",✔️ [True]
2,What is the role of the GST Network (GSTN)?,"GSTN would provide three front end services to the taxpayers namely registration, payment and return. It would be developing back-end IT modules for 27 States...",['Chap. 1 GST — Concept & Status 31 \n 27. Goods & Services Tax Network \n27.1 Goods and Services Tax Network (GSTN) has been set...,"The role of the GST Network (GSTN) is to provide front-end services to taxpayers, including registration, payment, and return filing, and to develop back-end IT...",False
3,What is the late fees per day for not filing Anual GST returns?,The penalty for not filing GST returns is ₹100 per day for CGST and ₹100 per day for SGST,"['976 GST Smart Guide Chap. 54 \n• Amount of pre-deposit payable for filing of appeal under the CGST \nAct, 2017 before the Appellate Authority and...",The late fees per day for not filing Annual GST returns is ₹200 (₹100 for CGST and ₹100 for SGST).,False
4,What is the GST compensation cess?,GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.,"['of the Council, for the purposes of providing compensation to the States for \nloss of revenue arising on account of implementation of the goods and...",GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.,False
5,What are the components of GST?,"The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).",['2 GST Smart Guide Chap. 1 \n principal causes of the prosperity of Great Britain; every great country being \nnecessarily the best and most extensive...,"The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).",False
6,What is a non-resident taxable person under GST?,A non-resident taxable person is someone who occasionally supplies goods or services in India without having a fixed place of business and is liable to...,"['(g) any casual taxable \nperson located in the \ntaxable territory.', 'various taxable person as under:', 'registration indicated in his application of registration, an application in...",A non,False


28.57

#### Defining custom metric

In [35]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of the answer"""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc = "Yes or No")

In [36]:
def metric(gold, pred, trace = None):
    question, answer, ans = gold.question, gold.answer, pred.answer

    engaging = "Does the assessed text make for a self-contained, engaging answer?"
    correct = f"The text should answer `{question}` with `{answer}`. Does the assessed text contain this answer?"
    
    with dspy.context(lm=lm_model):
        correct =  dspy.Predict(Assess)(assessed_text = ans, assessment_question = correct)
        engaging = dspy.Predict(Assess)(assessed_text = ans, assessment_question = engaging)

    correct, engaging = [m.assessment_answer.lower() == 'yes' for m in [correct, engaging]]
    score = (correct + engaging) if correct else 0

    if trace is not None: return score >= 2
    return score / 2.0

In [37]:
# Getting the accuracy with the customized model
scores = []
for x in testset:
    pred = rag(**x.inputs())
    print(x, pred.answer)
    score = metric(x, pred)
    scores.append(score)
    
print(f"Accuracy: {np.mean(scores)*100}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.13it/s]


Example({'question': 'What is MSME?', 'answer': 'MSME stands for Micro, Small and Medium Enterprises'}) (input_keys={'question'}) Micro, Small and Medium Enterprises


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.49it/s]


Example({'question': 'What are the benefits to MSMEs under GST?', 'answer': 'The Micro, Small and Medium Enterprises have been accorded with lot of benefits in terms of compliance reliefs given in the form of threshold exemptions, Composition levy scheme, quarterly filing of the GST returns to mention a few. While doing so, it has also been kept in mind that they do not become uncompetitive even as they are given all the benefits of GST.'}) (input_keys={'question'}) The benefits to MSMEs under GST include compliance reliefs like threshold exemptions, the Composition levy scheme, and quarterly filing of GST returns.


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.34it/s]


...

Example({'question': 'What is the GST compensation cess?', 'answer': 'GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.'}) (input_keys={'question'}) The GST compensation cess is an additional levy on certain notified goods, over and above the applicable GST, aimed at compensating states for revenue loss due to the implementation of GST, effective from July 1, 2017, for a period of five years or more as recommended by the GST Council.


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.13it/s]


Example({'question': 'What are the components of GST?', 'answer': 'The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).'}) (input_keys={'question'}) CGST, SGST/UTGST, and IGST
Backing off 0.5 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {}
Backing off 1.8 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {}
Backing off 3.9 seconds after 3 tries calling function <function AzureOpenAI.request at 0x1a19d4cc0> with kwargs {}


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.61it/s]


Example({'question': 'What is a non-resident taxable person under GST?', 'answer': 'A non-resident taxable person is someone who occasionally supplies goods or services in India without having a fixed place of business and is liable to pay GST.'}) (input_keys={'question'}) A non-resident taxable person under GST is any person who occasionally undertakes transactions involving the supply of goods or services or both.
Accuracy: 42.857142857142854


In [38]:
# Getting the accuracy of new optimized model on custom metric
scores = []
for x in testset:
    pred = compiled_rag(**x.inputs())
    print(x, pred.answer)
    score = metric(x, pred)
    scores.append(score)
    
print(f"Accuracy: {np.mean(scores)*100}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 23.55it/s]

Example({'question': 'What is MSME?', 'answer': 'MSME stands for Micro, Small and Medium Enterprises'}) (input_keys={'question'}) MSME stands for Micro, Small, and Medium Enterprises.



Batches: 100%|██████████| 1/1 [00:00<00:00, 16.97it/s]


Example({'question': 'What are the benefits to MSMEs under GST?', 'answer': 'The Micro, Small and Medium Enterprises have been accorded with lot of benefits in terms of compliance reliefs given in the form of threshold exemptions, Composition levy scheme, quarterly filing of the GST returns to mention a few. While doing so, it has also been kept in mind that they do not become uncompetitive even as they are given all the benefits of GST.'}) (input_keys={'question'}) The benefits to MSMEs under GST include compliance reliefs such as threshold exemptions, the Composition levy scheme, and the provision for quarterly filing of GST returns.


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.12it/s]


...

Example({'question': 'What is the GST compensation cess?', 'answer': 'GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.'}) (input_keys={'question'}) GST compensation cess is levied on certain goods to compensate states for revenue loss due to the implementation of GST.


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.98it/s]


Example({'question': 'What are the components of GST?', 'answer': 'The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).'}) (input_keys={'question'}) The components of GST are Central GST (CGST), State GST (SGST), and Integrated GST (IGST).


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.98it/s]


Example({'question': 'What is a non-resident taxable person under GST?', 'answer': 'A non-resident taxable person is someone who occasionally supplies goods or services in India without having a fixed place of business and is liable to pay GST.'}) (input_keys={'question'}) A non
Accuracy: 85.71428571428571


#### Running the RAG 

In [43]:
query = "What is GST?"
compiled_rag(query).answer

Batches: 100%|██████████| 1/1 [00:00<00:00, 27.15it/s]


'GST stands for Goods and Services Tax, a comprehensive, multi-stage, destination-based tax levied on every value addition.'

In [44]:
query = "Who needs to file GST?"
compiled_rag(query).answer

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.61it/s]


'Persons making any inter-State taxable supply, casual taxable persons'

In [45]:
query = "What is the penalty if someone misses to file GST?"
compiled_rag(query).answer

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.78it/s]


'The penalty for not filing GST returns is ₹100 per day for CGST and ₹100 per day for SGST.'